In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [15]:
##With 4 steps## 
data = pd.read_csv('Final.csv')
Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)
grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))
data = grouped_lists[['Albumin/Creatinine_Res']]
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
for index, row in data.iterrows() :
    raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 4 :
        # define input sequence
# choose a number of time steps
        n_steps = 3
# split into samples
        X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
# fit model
        model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
        x_input = np.asarray(raw_seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat)
        y_actual = np.asarray(raw_seq[-1:])
        rms = sqrt(mean_squared_error(y_actual, yhat))
        print(rms)
    else :
        continue

[[100.67543]]
14.324569702148438
[[847.0017]]
0.001708984375
[[6.7320747]]
0.7320747375488281
[[555.96185]]
48.03814697265625
[[13.951241]]
0.04875946044921875
[[24.046623]]
0.04662322998046875
[[9.467572]]
0.4675722122192383
[[194.91467]]
26.0853271484375
[[1439.7635]]
3.7635498046875
[[56.352062]]
0.6479377746582031


[[30.597872]]
2.597871780395508
[[117.70578]]
0.705780029296875
[[500.15796]]
409.157958984375
[[62.03314]]
0.033138275146484375
[[383.32544]]
0.325439453125
[[25.984045]]
0.015954971313476562
[[71.024536]]
14.9754638671875
[[372.13632]]
5.863677978515625
[[378.0881]]
0.088104248046875
[[18.768091]]
1.7680912017822266
[[50.994987]]
0.00501251220703125


[[11.922851]]
0.9228506088256836
[[40660.406]]
7878.59375
[[28.352425]]
0.6475753784179688
[[13.092968]]
0.09296798706054688
[[33.12947]]
2.1294708251953125
[[25.26087]]
0.7391300201416016
[[318.56158]]
0.56158447265625
[[21.945635]]
3.0543651580810547
[[-404.80527]]
528.8052673339844
[[13.551828]]
1.551828384399414
[[10.967056]]
0.0329437255859375


[[154.00186]]
0.001861572265625
[[47.549828]]
0.45017242431640625
[[20.777218]]
3.2227821350097656
[[14.999491]]
0.0005092620849609375
[[52.666668]]
0.6666679382324219
[[28.848299]]
0.1517009735107422
[[26.499493]]
0.4994926452636719
[[77.33675]]
7.6632537841796875
[[31.244127]]
0.2441272735595703
[[26.2671]]
0.26709938049316406
[[124.27805]]
1.7219467163085938


[[767.7709]]
0.7708740234375
[[27.907658]]
1.0923423767089844
[[450.05838]]
9.058380126953125
[[190.9988]]
0.0012054443359375
[[281.79636]]
0.203643798828125
[[57.011517]]
0.011516571044921875
[[380.42188]]
2.578125
[[8.122125]]
0.8778753280639648
[[30.321562]]
0.6784381866455078
[[22.254234]]
0.25423431396484375
[[273.4397]]
3.560302734375


[[1846.0853]]
23.0853271484375
[[19.000666]]
0.0006656646728515625
[[413.6786]]
1.3214111328125
[[19.222836]]
2.7771644592285156
[[1264.8413]]
882.84130859375
[[62.027687]]
0.02768707275390625
[[24.575443]]
1.4245567321777344
[[1978.1227]]
124.1226806640625
[[25.002066]]
0.0020656585693359375
[[87.30231]]
1.30230712890625
[[86.68452]]
23.68451690673828


[[213.63573]]
1.3642730712890625
[[58.181694]]
0.18169403076171875
[[16.926807]]
0.07319259643554688
[[27.814074]]
0.1859264373779297
[[105.80841]]
7.19158935546875
[[407.0945]]
0.905487060546875
[[13.801506]]
0.8015060424804688
[[47.179104]]
61.82089614868164
[[1913.3569]]
997.64306640625
[[62.360435]]
0.6395645141601562
[[55.181522]]
0.18152236938476562


[[150.9992]]
0.00079345703125
[[51.27344]]
2.2734413146972656
[[28.50378]]
0.4962196350097656
[[61.45172]]
0.45172119140625
[[97.55058]]
0.5505828857421875
[[26.995571]]
0.004428863525390625
[[115.9544]]
0.04560089111328125
[[26.083158]]
0.9168415069580078
[[122.96674]]
0.03325653076171875


KeyboardInterrupt: 

In [11]:
##Better - 8 steps##
data = pd.read_csv('Final.csv')
Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)
grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))
data = grouped_lists[['Albumin/Creatinine_Res']]
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
for index, row in data.iterrows() :
    raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 8 :
        # define input sequence
# choose a number of time steps
        n_steps = 3
# split into samples
        X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
# fit model
        model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
        x_input = np.asarray(raw_seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat)
        y_actual = np.asarray(raw_seq[-1:])
        rms = sqrt(mean_squared_error(y_actual, yhat))
        print(rms)
    else :
        continue

[[173.43753]]
58.437530517578125
[[338.59912]]
265.40087890625
[[14.848296]]
0.8482961654663086
[[24.065527]]
0.06552696228027344
[[9.757224]]
0.7572240829467773
[[27.733976]]
193.26602363586426
[[50.67803]]
6.321971893310547
[[27.912403]]
0.08759689331054688
[[754.3494]]
663.3494262695312
[[62.00056]]
0.000560760498046875


[[385.3778]]
7.3778076171875
[[29.20793]]
0.2079296112060547
[[28.52082]]
2.4791793823242188
[[25.49445]]
0.5055503845214844
[[316.9668]]
192.966796875
[[52.06917]]
0.06917190551757812
[[126.30258]]
0.302581787109375
[[449.96906]]
8.96905517578125
[[282.1509]]
0.150909423828125
[[381.19867]]
1.80133056640625
[[26.136435]]
4.863565444946289


[[22.121191]]
0.12119102478027344
[[828.2041]]
446.2041015625
[[25.777037]]
0.2229633331298828
[[1876.2786]]
22.278564453125
[[64.27082]]
1.2708206176757812
[[215.16646]]
0.1664581298828125
[[16.796993]]
0.20300674438476562
[[27.923819]]
0.07618141174316406
[[1965.4937]]
945.50634765625
[[62.704895]]
0.29510498046875
[[55.03145]]
0.0314483642578125


[[29.15271]]
0.1527099609375
[[60.94891]]
0.051090240478515625
[[97.49288]]
0.49288177490234375
[[26.65523]]
0.3447704315185547
[[369.26096]]
341.2609558105469
[[28.4366]]
0.5634002685546875
[[24905.932]]
5300.068359375
[[24.90336]]
2.903360366821289
[[125.96898]]
0.0310211181640625
[[20.564137]]
1.5641365051269531
[[715.7865]]
470.7864990234375


[[33.27099]]
4.270988464355469
[[125.776985]]
0.22301483154296875
[[458.99686]]
11.996856689453125
[[27.017143]]
0.01714324951171875
[[182.18777]]
4.812225341796875
[[28.986832]]
0.9868316650390625
[[478.91962]]
0.08038330078125
[[61.967766]]
30.03223419189453
[[413.48016]]
2.48016357421875
[[89.47888]]
0.4788818359375
[[235.92873]]
108.92872619628906


[[27233.24]]
26819.240234375
[[632.94073]]
6.05926513671875
[[30.669579]]
0.33042144775390625


KeyboardInterrupt: 